In [1]:
#our usual importing
import pandas as pd
import numpy as np
import os

In [2]:
#and we make the path
path = r'C:\Users\Admin\Documents\Dec 22 Instacart Analysis'

In [3]:
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged.pkl'))

In [4]:
#question 2
df_ords_prods_merged.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

well for one, this is going to have every department, not just a few of them, so that will be more useful. It's weird that you can take a million rows and still only have not even half the department represented but that probably comes down to how the data is organized in the table I'd bet. the low mean we saw for department 17 has also raised significantly so that would possibly impact our analysis. and on the same token, the means for the highest ranking categories in the exercise have all decreased. a random sample, rather than a certain group of rows, would probably be better for analytical purposes.

In [6]:
#next few rows are from the exercise/answering question 4
df_ords_prods_merged['max_order'] = df_ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merged.loc[(df_ords_prods_merged['max_order'] <= 40) & (df_ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
#for question 5, we use our group by function on the loyalty flag, then take the mean of the prices
df_ords_prods_merged.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal customer       9.945181
New customer        12.851975
Regular customer    12.053843
Name: prices, dtype: float64

at the same time, i wonder if there's a difference in order total between the loyalty flag tiers. Like maybe they're buying cheaper products but more of them. 

In [10]:
#we gotta make our new column that groups by order id and returns the sum of the prices as a new column
df_ords_prods_merged['order_cost'] = df_ords_prods_merged.groupby(['order_id'])['prices'].transform(np.sum)

In [11]:
#checking to see that the column is there and things look alright at a glance
df_ords_prods_merged[['order_id', 'order_cost']].head(10)

,order_id,order_cost
0,2539329,30
1,2398795,34
2,473747,27
3,2254736,25
4,431534,55
5,3367565,24
6,550135,28
7,3108588,38
8,2295261,38
9,2550362,60


In [12]:
#calling a certain order id to see if the sum of the prices matches what's in the order cost column
df_ords_prods_merged.loc[df_ords_prods_merged['order_id']==2539329]

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,order_cost
0,0,2539329,1,1,2,8,NaN,196,1,0,...,77,7,9,both,Mid-range product,Regularly busy,Average orders,10,New customer,30
35791,0,2539329,1,1,2,8,NaN,14084,2,0,...,91,16,12,both,Mid-range product,Regularly busy,Average orders,10,New customer,30
51726,0,2539329,1,1,2,8,NaN,12427,3,0,...,23,19,4,both,Low-range product,Regularly busy,Average orders,10,New customer,30
58202,0,2539329,1,1,2,8,NaN,26088,4,0,...,23,19,4,both,Low-range product,Regularly busy,Average orders,10,New customer,30
60725,0,2539329,1,1,2,8,NaN,26405,5,0,...,54,17,1,both,Low-range product,Regularly busy,Average orders,10,New customer,30


OMG IT WORKED

In [13]:
#now we group by the loyalty flag and take the average order cost
df_ords_prods_merged.groupby('loyalty_flag')['order_cost'].mean()

loyalty_flag
Loyal customer      151.924147
New customer        198.603838
Regular customer    194.491413
Name: order_cost, dtype: float64

well there is our answer, the more you order the less you tend to spend, on both counts. now i wonder if it's because they order more frequently? 

In [15]:
#LET'S FIND OUT SHALL WE
df_ords_prods_merged.groupby('loyalty_flag')['days_since_last_order'].mean()

loyalty_flag
Loyal customer       6.026656
New customer        17.976936
Regular customer    12.155135
Name: days_since_last_order, dtype: float64

that seems to be it. they have smaller orders but do it more frequently.

In [18]:
#moving onto question 6. here we are making our new column that returns the average amount spent per item, based on the user id
df_ords_prods_merged['avg_spent_per_item'] = df_ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [19]:
#time to make our user defined function!
df_ords_prods_merged.loc[df_ords_prods_merged['avg_spent_per_item'] >= 10, 'spender_type'] = 'High spender'
df_ords_prods_merged.loc[df_ords_prods_merged['avg_spent_per_item'] < 10, 'spender_type'] = 'Low spender'

In [20]:
#now let's see if it worked
df_ords_prods_merged.head(10)

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,order_cost,avg_spent_per_item,spender_type
0,0,2539329,1,1,2,8,NaN,196,1,0,...,9,both,Mid-range product,Regularly busy,Average orders,10,New customer,30,6.084746,Low spender
1,1,2398795,1,2,3,7,15.0,196,1,1,...,9,both,Mid-range product,Least busy days,Average orders,10,New customer,34,6.084746,Low spender
2,2,473747,1,3,3,12,21.0,196,1,1,...,9,both,Mid-range product,Least busy days,Most orders,10,New customer,27,6.084746,Low spender
3,3,2254736,1,4,4,7,29.0,196,1,1,...,9,both,Mid-range product,Least busy days,Average orders,10,New customer,25,6.084746,Low spender
4,4,431534,1,5,4,15,28.0,196,1,1,...,9,both,Mid-range product,Least busy days,Most orders,10,New customer,55,6.084746,Low spender
5,5,3367565,1,6,2,7,19.0,196,1,1,...,9,both,Mid-range product,Regularly busy,Average orders,10,New customer,24,6.084746,Low spender
6,6,550135,1,7,1,9,20.0,196,1,1,...,9,both,Mid-range product,Busiest days,Average orders,10,New customer,28,6.084746,Low spender
7,7,3108588,1,8,1,14,14.0,196,2,1,...,9,both,Mid-range product,Busiest days,Most orders,10,New customer,38,6.084746,Low spender
8,8,2295261,1,9,1,16,0.0,196,4,1,...,9,both,Mid-range product,Busiest days,Average orders,10,New customer,38,6.084746,Low spender
9,9,2550362,1,10,4,8,30.0,196,1,1,...,9,both,Mid-range product,Least busy days,Average orders,10,New customer,60,6.084746,Low spender


it looks like everything works, but good lord this dataframe is getting massive

In [21]:
#question 7. again, we start by building our relevant column
df_ords_prods_merged['order_frequency'] = df_ords_prods_merged.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [22]:
#and then we make our user function
df_ords_prods_merged.loc[df_ords_prods_merged['order_frequency'] > 20, 'frequency_flag'] = 'Non frequent customer'
df_ords_prods_merged.loc[(df_ords_prods_merged['order_frequency'] <= 20) & (df_ords_prods_merged['order_frequency'] > 10), 'frequency_flag'] = 'Regular customer'
df_ords_prods_merged.loc[df_ords_prods_merged['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [23]:
#now we just double check and done!
df_ords_prods_merged.head(25)

,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,order_cost,avg_spent_per_item,spender_type,order_frequency,frequency_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Average orders,10,New customer,30,6.084746,Low spender,20.5,Non frequent customer
1,1,2398795,1,2,3,7,15.0,196,1,1,...,Mid-range product,Least busy days,Average orders,10,New customer,34,6.084746,Low spender,20.5,Non frequent customer
2,2,473747,1,3,3,12,21.0,196,1,1,...,Mid-range product,Least busy days,Most orders,10,New customer,27,6.084746,Low spender,20.5,Non frequent customer
3,3,2254736,1,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy days,Average orders,10,New customer,25,6.084746,Low spender,20.5,Non frequent customer
4,4,431534,1,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy days,Most orders,10,New customer,55,6.084746,Low spender,20.5,Non frequent customer
5,5,3367565,1,6,2,7,19.0,196,1,1,...,Mid-range product,Regularly busy,Average orders,10,New customer,24,6.084746,Low spender,20.5,Non frequent customer
6,6,550135,1,7,1,9,20.0,196,1,1,...,Mid-range product,Busiest days,Average orders,10,New customer,28,6.084746,Low spender,20.5,Non frequent customer
7,7,3108588,1,8,1,14,14.0,196,2,1,...,Mid-range product,Busiest days,Most orders,10,New customer,38,6.084746,Low spender,20.5,Non frequent customer
8,8,2295261,1,9,1,16,0.0,196,4,1,...,Mid-range product,Busiest days,Average orders,10,New customer,38,6.084746,Low spender,20.5,Non frequent customer
9,9,2550362,1,10,4,8,30.0,196,1,1,...,Mid-range product,Least busy days,Average orders,10,New customer,60,6.084746,Low spender,20.5,Non frequent customer


In [24]:
df_ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_flagged.pkl'))